# REFRAG v2 - Training Pipeline for Google Colab

<a href="https://colab.research.google.com/github/khanhtimn/Project-III/blob/main/refrag_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook implements the full REFRAG v2 training pipeline, designed to run on Google Colab with GPU acceleration.

## Pipeline Overview

1. **Setup**: Install dependencies and configure environment
2. **Data Preparation**: Upload or download training data
3. **Stage 0**: Build FAISS index from corpus
4. **Stage 1**: Reconstruction training (freeze decoder, train encoder + projector)
5. **Stage 2**: Continual Pre-Training (CPT) - unfreeze decoder
6. **Stage 3**: Policy training (optional, GRPO-style)
7. **Evaluation**: Test generation and compare models

## REFRAG Paper Summary

REFRAG (Retrieval-Enhanced Framework for RAG) compresses retrieved passages into dense embeddings:
- Compresses k tokens into 1 embedding (k=16 recommended: 30× speedup, -0.3% accuracy)
- Uses RoBERTa-Large encoder + LLaMA decoder
- 9-stage curriculum learning for training
- Optional selective expansion policy via GRPO

In [ ]:
# @title 🔧 Environment Setup { display-mode: "form" }
# @markdown Check GPU availability and setup environment

import subprocess
import sys

# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("⚠️ Not running in Colab - some features may differ")

# Check GPU
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
    DEVICE = "cuda"
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("✅ MPS (Apple Silicon) Available")
    DEVICE = "mps"
else:
    print("⚠️ No GPU detected - training will be slow")
    DEVICE = "cpu"

print(f"\n📍 Device: {DEVICE}")
print(f"🐍 Python: {sys.version.split()[0]}")
print(f"🔥 PyTorch: {torch.__version__}")

hello


In [ ]:
# @title 📦 Install Dependencies { display-mode: "form" }
# @markdown Install all required packages for REFRAG v2

import subprocess
import sys

def install_packages():
    """Install required packages for REFRAG v2."""
    packages = [
        "torch>=2.0.0",
        "transformers>=4.40.0",
        "accelerate>=0.20.0",
        "sentencepiece>=0.1.99",
        "safetensors>=0.4.0",
        "faiss-cpu>=1.7.4",  # Use faiss-gpu on Colab if available
        "sentence-transformers>=2.2.0",
        "datasets>=2.14.0",
        "tqdm>=4.65.0",
        "numpy>=1.24.0",
    ]
    
    print("📦 Installing dependencies...")
    for pkg in packages:
        pkg_name = pkg.split(">=")[0].split("==")[0]
        try:
            __import__(pkg_name.replace("-", "_"))
            print(f"  ✅ {pkg_name} already installed")
        except ImportError:
            print(f"  📥 Installing {pkg}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    
    # Try to install faiss-gpu on Colab (falls back to cpu)
    try:
        import google.colab
        print("  📥 Attempting faiss-gpu for Colab...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "faiss-gpu"])
        print("  ✅ faiss-gpu installed")
    except:
        print("  ℹ️ Using faiss-cpu")
    
    print("\n✅ All dependencies installed!")

install_packages()

# Verify key imports
import torch
import transformers
import faiss
print(f"\n🔍 Verification:")
print(f"   torch: {torch.__version__}")
print(f"   transformers: {transformers.__version__}")
print(f"   faiss: {faiss.__version__ if hasattr(faiss, '__version__') else 'installed'}")

## 🔑 Hugging Face Authentication

You need to authenticate with Hugging Face to access gated models like LLaMA. 
Get your token from: https://huggingface.co/settings/tokens

In [ ]:
# @title 🔐 HuggingFace Login { display-mode: "form" }
# @markdown Enter your HuggingFace token (required for LLaMA models)

from huggingface_hub import login, whoami

# Check if already logged in
try:
    user_info = whoami()
    print(f"✅ Already logged in as: {user_info['name']}")
except:
    print("🔐 Please login to HuggingFace...")
    try:
        # For Colab, use notebook_login for interactive widget
        from google.colab import userdata
        try:
            # Try to get token from Colab secrets
            hf_token = userdata.get('HF_TOKEN')
            login(token=hf_token)
            print("✅ Logged in using Colab secret!")
        except:
            # Fall back to interactive login
            from huggingface_hub import notebook_login
            notebook_login()
    except ImportError:
        # Not in Colab, use standard login
        login()
    
    # Verify login
    user_info = whoami()
    print(f"✅ Logged in as: {user_info['name']}")

## ⚙️ Configuration

Configure the training parameters below. Adjust based on your GPU memory:
- **T4 (16GB)**: Use smaller batch size (2-4), k=16, smaller decoder model
- **A100 (40GB)**: Can use batch size 8, k=16, full LLaMA-3B
- **L4/A10 (24GB)**: Moderate settings

In [ ]:
# @title ⚙️ Training Configuration { display-mode: "form" }
# @markdown ### Model Configuration
ENCODER_MODEL = "roberta-large"  # @param ["roberta-large", "roberta-base", "distilroberta-base"]
DECODER_MODEL = "meta-llama/Llama-3.2-1B"  # @param ["meta-llama/Llama-3.2-1B", "meta-llama/Llama-3.2-3B", "TinyLlama/TinyLlama-1.1B-Chat-v1.0", "microsoft/phi-2"]
EMBED_MODEL = "BAAI/bge-small-en-v1.5"  # @param ["BAAI/bge-small-en-v1.5", "sentence-transformers/all-MiniLM-L6-v2"]

# @markdown ### Compression Settings
K = 16  # @param {type:"slider", min:8, max:64, step:8}
# @markdown K is the compression ratio (tokens per embedding). k=16 recommended.

# @markdown ### Training Hyperparameters
BATCH_SIZE = 4  # @param {type:"slider", min:1, max:16, step:1}
NUM_CURRICULUM_STAGES = 5  # @param {type:"slider", min:1, max:9, step:1}
# @markdown Reduce stages for faster training (9 is paper default)

LR_RECONSTRUCTION = 2e-4  # @param {type:"number"}
LR_CPT = 1e-5  # @param {type:"number"}
LR_CPT_DECODER = 5e-7  # @param {type:"number"}

# @markdown ### Retrieval Settings  
TOP_K = 4  # @param {type:"slider", min:1, max:10, step:1}

# @markdown ### Advanced Settings
USE_FP16 = False  # @param {type:"boolean"}
# @markdown FP16 can cause NaN issues - disable if training becomes unstable
SEED = 1337  # @param {type:"integer"}

# Print configuration summary
print("=" * 50)
print("📋 REFRAG v2 Configuration")
print("=" * 50)
print(f"Encoder:     {ENCODER_MODEL}")
print(f"Decoder:     {DECODER_MODEL}")
print(f"Embed Model: {EMBED_MODEL}")
print(f"K (chunk):   {K}")
print(f"Batch Size:  {BATCH_SIZE}")
print(f"Stages:      {NUM_CURRICULUM_STAGES}")
print(f"FP16:        {USE_FP16}")
print("=" * 50)

## 📂 Data Preparation

You can either:
1. **Upload your own data** - Upload files to `data/` directory
2. **Use sample data** - Generate synthetic training data
3. **Download from HuggingFace** - Load datasets from the Hub

In [ ]:
# @title 📂 Setup Data Directories { display-mode: "form" }
import os
import json

# Create directory structure
DATA_DIR = "data"
RUNS_DIR = "runs"
INDEX_DIR = os.path.join(RUNS_DIR, "index")
RECON_DIR = os.path.join(RUNS_DIR, "refrag_v2_recon")
CPT_DIR = os.path.join(RUNS_DIR, "refrag_v2_cpt")
POLICY_DIR = os.path.join(RUNS_DIR, "refrag_v2_policy")

for d in [DATA_DIR, INDEX_DIR, RECON_DIR, CPT_DIR, POLICY_DIR]:
    os.makedirs(d, exist_ok=True)
    print(f"✅ Created: {d}")

# File paths
CORPUS_FILE = os.path.join(DATA_DIR, "corpus.txt")
CPT_TRAIN_FILE = os.path.join(DATA_DIR, "cpt_train.jsonl")
RAG_TRAIN_FILE = os.path.join(DATA_DIR, "rag_train.jsonl")
RAG_EVAL_FILE = os.path.join(DATA_DIR, "rag_eval.jsonl")

print(f"\n📁 Data directory: {os.path.abspath(DATA_DIR)}")
print(f"📁 Runs directory: {os.path.abspath(RUNS_DIR)}")

In [ ]:
# @title 📤 Option 1: Upload Your Data (Colab) { display-mode: "form" }
# @markdown Run this cell to upload your own data files

try:
    from google.colab import files
    
    print("📤 Upload your data files:")
    print("   - corpus.txt (one passage per line)")
    print("   - cpt_train.jsonl (training data with 'text' field)")
    print("   - rag_train.jsonl (QA pairs with 'question' and 'answers' fields)")
    print("\n⏳ Waiting for file upload...")
    
    uploaded = files.upload()
    
    for filename, content in uploaded.items():
        filepath = os.path.join(DATA_DIR, filename)
        with open(filepath, 'wb') as f:
            f.write(content)
        print(f"✅ Saved: {filepath} ({len(content)} bytes)")
        
except ImportError:
    print("ℹ️ Not running in Colab. Place your data files in the 'data/' directory manually.")
except Exception as e:
    print(f"⚠️ Upload cancelled or failed: {e}")

In [ ]:
# @title 🔧 Option 2: Generate Sample Data { display-mode: "form" }
# @markdown Generate synthetic training data for testing the pipeline

import random

def generate_sample_data(num_passages=500, num_qa=200):
    """Generate synthetic training data for REFRAG."""
    
    # Generate corpus (passages)
    passages = []
    for i in range(num_passages):
        city_id = i % 100
        country_id = i % 50
        river_id = i * 2
        year = 1200 + (i * 3) % 800
        pop = 50000 + i * 500
        
        passage = f"TITLE: Atlas Entry — City_{city_id}. City_{city_id} is a regional hub in Country_{country_id:03d}. " \
                  f"It lies on the banks of River_{river_id:03d} and was founded in {year}. " \
                  f"The metropolitan population is approximately {pop}. " \
                  f"The primary research institution is University_{i % 200:03d}. " \
                  f"City_{city_id} specializes in Industry_{i % 100:03d} and advanced logistics."
        passages.append(passage)
    
    # Add some variety - materials and biographies
    for i in range(num_passages // 4):
        alloy_passage = f"TITLE: Encyclopedia of Materials — Alloy_{i}. " \
                       f"Composition: Metal_{i % 100:02d} {60 + i % 20}%, Additive_{i % 50:02d} {30 - i % 10}%. " \
                       f"Melting point: {900 + i * 3} °C. Yield strengths near {200 + i * 5} MPa."
        passages.append(alloy_passage)
        
        bio_passage = f"TITLE: Biography — Person_{i}. Person_{i} is a researcher in Field_{i % 50:02d} " \
                     f"affiliated with Institution_{i % 200:03d}. In {1950 + i % 60}, they proposed Contribution_{i:04d}."
        passages.append(bio_passage)
    
    # Write corpus
    with open(CORPUS_FILE, 'w', encoding='utf-8') as f:
        for p in passages:
            f.write(p + '\n')
    print(f"✅ Generated corpus: {len(passages)} passages -> {CORPUS_FILE}")
    
    # Generate CPT training data (longer text for reconstruction)
    cpt_data = []
    for i in range(min(len(passages), 400)):
        # Combine a few passages for longer context
        combined = " ".join(passages[i:i+3])
        cpt_data.append({"id": f"cpt-{i}", "text": combined})
    
    with open(CPT_TRAIN_FILE, 'w', encoding='utf-8') as f:
        for item in cpt_data:
            f.write(json.dumps(item) + '\n')
    print(f"✅ Generated CPT training data: {len(cpt_data)} examples -> {CPT_TRAIN_FILE}")
    
    # Generate RAG QA data
    qa_data = []
    for i in range(num_qa):
        city_id = i % 100
        q_type = i % 4
        
        if q_type == 0:
            question = f"Which river flows through City_{city_id}?"
            answer = f"River_{city_id * 2:03d}"
        elif q_type == 1:
            question = f"What is the primary research institution in City_{city_id}?"
            answer = f"University_{city_id % 200:03d}"
        elif q_type == 2:
            question = f"What is the melting point of Alloy_{city_id}?"
            answer = f"{900 + city_id * 3} °C"
        else:
            question = f"In which field does Person_{city_id} work?"
            answer = f"Field_{city_id % 50:02d}"
        
        qa_data.append({
            "id": f"qa-{i}",
            "question": question,
            "answers": [answer]
        })
    
    # Split into train and eval
    random.shuffle(qa_data)
    split_idx = int(len(qa_data) * 0.8)
    train_qa = qa_data[:split_idx]
    eval_qa = qa_data[split_idx:]
    
    with open(RAG_TRAIN_FILE, 'w', encoding='utf-8') as f:
        for item in train_qa:
            f.write(json.dumps(item) + '\n')
    print(f"✅ Generated RAG training data: {len(train_qa)} examples -> {RAG_TRAIN_FILE}")
    
    with open(RAG_EVAL_FILE, 'w', encoding='utf-8') as f:
        for item in eval_qa:
            f.write(json.dumps(item) + '\n')
    print(f"✅ Generated RAG eval data: {len(eval_qa)} examples -> {RAG_EVAL_FILE}")
    
    return len(passages), len(cpt_data), len(train_qa), len(eval_qa)

# Generate data
n_passages, n_cpt, n_train, n_eval = generate_sample_data()
print(f"\n📊 Data Summary:")
print(f"   Corpus passages: {n_passages}")
print(f"   CPT examples: {n_cpt}")
print(f"   RAG train QA: {n_train}")
print(f"   RAG eval QA: {n_eval}")

## 🏗️ REFRAG v2 Model Implementation

The following cells contain the complete REFRAG v2 implementation:
- Configuration dataclass
- Encoder, Projector, and Policy networks
- Full REFRAG model with training methods
- Training loops for all three stages

In [ ]:
# @title 🏗️ REFRAG v2 Core Implementation { display-mode: "form" }
# @markdown This cell contains the complete REFRAG v2 model implementation

import os
import json
import time
import random
import logging
from dataclasses import dataclass, asdict
from typing import List, Tuple, Dict, Optional, Any
from datetime import datetime
from tqdm.auto import tqdm

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    get_cosine_schedule_with_warmup,
)

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s')
logger = logging.getLogger(__name__)

# ============================================================================
# Configuration
# ============================================================================

@dataclass
class REFRAGConfig:
    """Configuration for REFRAG v2 model."""
    encoder_name: str = ENCODER_MODEL
    decoder_name: str = DECODER_MODEL
    chunk_size_k: int = K
    db_chunk_size: int = 256
    max_context_tokens: int = 128
    max_output_tokens: int = 128
    max_query_tokens: int = 256
    expansion_fraction_p: float = 0.1
    batch_size: int = BATCH_SIZE
    gradient_accumulation_steps: int = 32
    lr_reconstruction: float = LR_RECONSTRUCTION
    lr_cpt: float = LR_CPT
    lr_cpt_decoder: float = LR_CPT_DECODER
    lr_finetune: float = 2e-5
    lr_policy: float = 1e-4
    weight_decay: float = 0.0
    warmup_ratio: float = 0.04
    max_grad_norm: float = 1.0
    num_curriculum_stages: int = NUM_CURRICULUM_STAGES
    epochs_per_stage: int = 1
    policy_hidden_dim: int = 256
    grpo_group_size: int = 4
    ppo_clip_epsilon: float = 0.2
    fp16: bool = USE_FP16
    seed: int = SEED

    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)


# ============================================================================
# Utilities
# ============================================================================

def seed_everything(seed: int):
    """Set all random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if torch.cuda.is_available():
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


def get_device():
    """Get the best available device."""
    if torch.cuda.is_available():
        return torch.device('cuda')
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        return torch.device('mps')
    return torch.device('cpu')


# ============================================================================
# Curriculum Learning Schedule
# ============================================================================

def get_curriculum_schedule(num_stages: int = 9, max_chunks: int = 256) -> List[Dict]:
    """Generate the curriculum schedule from the paper."""
    chunk_multipliers = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    schedule = []

    for stage in range(num_stages):
        stage_config = {
            'stage': stage + 1,
            'max_chunks': min(chunk_multipliers[stage] if stage < len(chunk_multipliers) else max_chunks, max_chunks),
            'weights': {}
        }

        for j, mult in enumerate(chunk_multipliers):
            if mult > max_chunks:
                continue
            if j <= stage:
                weight = 0.5 ** (stage - j)
            else:
                weight = 0.5 ** (j - stage) * 0.1
            stage_config['weights'][mult] = weight

        total = sum(stage_config['weights'].values())
        for k in stage_config['weights']:
            stage_config['weights'][k] /= total

        schedule.append(stage_config)

    return schedule


def sample_num_chunks_for_stage(stage_config: Dict) -> int:
    """Sample number of chunks based on stage weights."""
    chunks = list(stage_config['weights'].keys())
    weights = list(stage_config['weights'].values())
    return random.choices(chunks, weights=weights, k=1)[0]


# ============================================================================
# Data Loading
# ============================================================================

class CPTDataset(Dataset):
    """Dataset for Continual Pre-Training."""

    def __init__(self, data_path: str, tokenizer, max_length: int = 4096):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = []

        if os.path.exists(data_path):
            with open(data_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        item = json.loads(line)
                        if 'text' in item or 'tokens' in item:
                            self.data.append(item)
        logger.info(f"Loaded {len(self.data)} examples from {data_path}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item.get('text', item.get('tokens', ''))
        return {'text': text, 'id': item.get('id', str(idx))}


class RAGDataset(Dataset):
    """Dataset for RAG training/evaluation."""

    def __init__(self, data_path: str):
        self.data = []
        if os.path.exists(data_path):
            with open(data_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        item = json.loads(line)
                        if 'question' in item:
                            self.data.append(item)
        logger.info(f"Loaded {len(self.data)} QA examples from {data_path}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


print("✅ Core utilities loaded")

In [ ]:
# @title 🔍 Passage Retriever (FAISS) { display-mode: "form" }
# @markdown FAISS-based passage retriever for building and searching the index

import faiss

class PassageRetriever:
    """FAISS-based passage retriever."""

    def __init__(self, model_name: str = EMBED_MODEL, device=None):
        self.device = device or get_device()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.model.eval()
        self.embed_dim = self.model.config.hidden_size
        self.index = None
        self.passages = []

    @torch.no_grad()
    def encode(self, texts: List[str], batch_size: int = 32) -> np.ndarray:
        """Encode texts to vectors."""
        all_embeddings = []

        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding", leave=False):
            batch = texts[i:i + batch_size]
            inputs = self.tokenizer(
                batch, padding=True, truncation=True, max_length=512, return_tensors='pt'
            ).to(self.device)

            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]  # CLS pooling
            embeddings = F.normalize(embeddings, dim=-1)
            all_embeddings.append(embeddings.cpu().numpy())

        return np.vstack(all_embeddings).astype(np.float32)

    def build_index(self, passages: List[str], index_path: str):
        """Build and save FAISS index."""
        self.passages = passages
        print(f"📊 Encoding {len(passages)} passages...")
        embeddings = self.encode(passages)

        faiss.normalize_L2(embeddings)
        self.index = faiss.IndexFlatIP(self.embed_dim)
        self.index.add(embeddings)

        os.makedirs(os.path.dirname(index_path) or '.', exist_ok=True)
        faiss.write_index(self.index, index_path)
        np.save(index_path.replace('.index', '_passages.npy'), np.array(passages, dtype=object))

        print(f"✅ Built index with {len(passages)} passages -> {index_path}")

    def load_index(self, index_path: str):
        """Load FAISS index."""
        self.index = faiss.read_index(index_path)
        passages_path = index_path.replace('.index', '_passages.npy')
        self.passages = np.load(passages_path, allow_pickle=True).tolist()
        print(f"✅ Loaded index with {len(self.passages)} passages")

    def search(self, query: str, top_k: int = 10) -> List[Tuple[str, float]]:
        """Search for relevant passages."""
        if self.index is None:
            raise ValueError("Index not loaded")

        query_vec = self.encode([query])
        faiss.normalize_L2(query_vec)
        scores, indices = self.index.search(query_vec, top_k)

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx >= 0 and idx < len(self.passages):
                results.append((self.passages[idx], float(score)))
        return results


print("✅ Passage retriever loaded")

In [ ]:
# @title 🧠 REFRAG Model Components { display-mode: "form" }
# @markdown Encoder, Projector, and Policy network definitions

class ChunkEncoder(nn.Module):
    """Encoder for chunk embeddings using RoBERTa."""

    def __init__(self, model_name: str = "roberta-large"):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.hidden_size = self.model.config.hidden_size

    def forward(self, texts: List[str], device=None) -> torch.Tensor:
        """Encode text chunks to embeddings. Returns: [num_chunks, hidden_size]"""
        if len(texts) == 0:
            return torch.zeros(0, self.hidden_size, device=device)

        device = device or next(self.model.parameters()).device
        inputs = self.tokenizer(
            texts, padding=True, truncation=True, max_length=512, return_tensors='pt'
        ).to(device)

        outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # CLS pooling
        return F.normalize(embeddings, dim=-1)


class ProjectionLayer(nn.Module):
    """Projection layer φ: encoder_dim → decoder_dim"""

    def __init__(self, encoder_dim: int, decoder_dim: int):
        super().__init__()
        self.projection = nn.Sequential(
            nn.Linear(encoder_dim, decoder_dim),
            nn.GELU(),
            nn.Linear(decoder_dim, decoder_dim),
        )
        self.scale = nn.Parameter(torch.ones(1))
        self.shift = nn.Parameter(torch.zeros(1))

    def forward(self, x: torch.Tensor, target_stats: Optional[Tuple[float, float]] = None) -> torch.Tensor:
        out = self.projection(x)
        out = out * self.scale + self.shift
        if target_stats is not None:
            target_mean, target_std = target_stats
            out_mean = out.mean()
            out_std = out.std() + 1e-8
            out = (out - out_mean) / out_std * target_std + target_mean
        return out


class ExpansionPolicy(nn.Module):
    """RL policy network for selective chunk expansion."""

    def __init__(self, input_dim: int, hidden_dim: int = 256):
        super().__init__()
        self.embedding_proj = nn.Linear(input_dim, hidden_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim, nhead=4, dim_feedforward=hidden_dim * 4,
                dropout=0.1, batch_first=True
            ),
            num_layers=2
        )
        self.output_head = nn.Linear(hidden_dim, 1)

    def forward(self, chunk_embeddings: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        squeeze = False
        if chunk_embeddings.dim() == 2:
            chunk_embeddings = chunk_embeddings.unsqueeze(0)
            squeeze = True

        x = self.embedding_proj(chunk_embeddings)
        x = self.transformer(x, src_key_padding_mask=mask)
        logits = self.output_head(x).squeeze(-1)

        if squeeze:
            logits = logits.squeeze(0)
        return logits

    def sample_expansion_mask(self, chunk_embeddings: torch.Tensor, max_expand_fraction: float = 0.1
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Sample which chunks to expand using the policy."""
        logits = self.forward(chunk_embeddings)
        probs = torch.sigmoid(logits)
        dist = torch.distributions.Bernoulli(probs=probs)
        samples = dist.sample()
        log_probs = dist.log_prob(samples)

        num_chunks = len(logits)
        max_expand = max(1, int(max_expand_fraction * num_chunks))

        if samples.sum() > max_expand:
            _, top_indices = torch.topk(logits, k=max_expand)
            new_samples = torch.zeros_like(samples)
            new_samples[top_indices] = 1
            samples = new_samples

        return samples.bool(), log_probs.sum()


print("✅ Model components loaded")

In [ ]:
# @title 🤖 Full REFRAG Model { display-mode: "form" }
# @markdown Complete REFRAG model with training and generation methods

class REFRAGModel(nn.Module):
    """Full REFRAG model implementing compress → sense → expand."""

    def __init__(self, config: REFRAGConfig):
        super().__init__()
        self.config = config
        self.device = get_device()

        # Initialize components
        print(f"📥 Loading encoder: {config.encoder_name}")
        self.encoder = ChunkEncoder(config.encoder_name)

        print(f"📥 Loading decoder: {config.decoder_name}")
        self.decoder_tokenizer = AutoTokenizer.from_pretrained(config.decoder_name)
        if self.decoder_tokenizer.pad_token is None:
            self.decoder_tokenizer.pad_token = self.decoder_tokenizer.eos_token

        # Determine dtype
        use_fp16 = config.fp16 and self.device.type not in ['mps', 'cpu']
        self.decoder = AutoModelForCausalLM.from_pretrained(
            config.decoder_name,
            torch_dtype=torch.float16 if use_fp16 else torch.float32,
        )

        self.decoder_embed_dim = self.decoder.get_input_embeddings().weight.shape[1]
        self.projector = ProjectionLayer(self.encoder.hidden_size, self.decoder_embed_dim)
        self.policy = ExpansionPolicy(self.encoder.hidden_size, config.policy_hidden_dim)

        self.eos_token_id = self.decoder_tokenizer.eos_token_id
        self.pad_token_id = self.decoder_tokenizer.pad_token_id

    def to(self, device):
        """Move model to device."""
        self.device = device
        self.encoder = self.encoder.to(device)
        self.decoder = self.decoder.to(device)
        self.projector = self.projector.to(device)
        self.policy = self.policy.to(device)
        return self

    def freeze_decoder(self):
        for param in self.decoder.parameters():
            param.requires_grad = False
        print("🔒 Decoder frozen")

    def unfreeze_decoder(self):
        for param in self.decoder.parameters():
            param.requires_grad = True
        print("🔓 Decoder unfrozen")

    def freeze_encoder_projector(self):
        for param in self.encoder.parameters():
            param.requires_grad = False
        for param in self.projector.parameters():
            param.requires_grad = False
        print("🔒 Encoder and projector frozen")

    def _tokenize_text(self, text: str, max_length: int) -> torch.Tensor:
        tokens = self.decoder_tokenizer(text, truncation=True, max_length=max_length, return_tensors='pt')
        return tokens.input_ids.to(self.device)

    def _get_decoder_embeddings(self, input_ids: torch.Tensor) -> torch.Tensor:
        return self.decoder.get_input_embeddings()(input_ids)

    def _chunk_tokens(self, input_ids: torch.Tensor, k: int) -> List[torch.Tensor]:
        ids = input_ids.squeeze(0) if input_ids.dim() > 1 else input_ids
        return [ids[i:i+k] for i in range(0, len(ids), k)]

    def _chunks_to_text(self, chunks: List[torch.Tensor]) -> List[str]:
        return [self.decoder_tokenizer.decode(c, skip_special_tokens=True) for c in chunks]

    def compute_reconstruction_loss(self, text: str, num_chunks_cap: Optional[int] = None) -> torch.Tensor:
        """Compute reconstruction loss for Stage 1 training."""
        k = self.config.chunk_size_k
        input_ids = self._tokenize_text(text, self.config.max_context_tokens)
        chunks = self._chunk_tokens(input_ids, k)
        if num_chunks_cap is not None:
            chunks = chunks[:num_chunks_cap]

        if len(chunks) == 0:
            return torch.tensor(0.0, device=self.device, requires_grad=True)

        chunk_texts = self._chunks_to_text(chunks)
        chunk_embeddings = self.encoder(chunk_texts, device=self.device)
        projected = self.projector(chunk_embeddings)

        total_loss = 0.0
        num_valid = 0

        for i, chunk_ids in enumerate(chunks):
            if len(chunk_ids) < 2:
                continue

            compressed_emb = projected[i:i+1, :]
            target_ids = chunk_ids.to(self.device)
            target_embs = self._get_decoder_embeddings(target_ids[:-1].unsqueeze(0))

            decoder_dtype = next(self.decoder.parameters()).dtype
            emb_to_concat = compressed_emb.unsqueeze(1)
            if decoder_dtype == torch.float16:
                emb_to_concat = emb_to_concat.clamp(-65000, 65000)
            emb_to_concat = emb_to_concat.to(dtype=decoder_dtype)

            input_embs = torch.cat([emb_to_concat, target_embs], dim=1)
            labels = torch.cat([torch.tensor([-100], device=self.device), target_ids]).unsqueeze(0)
            labels = labels[:, :input_embs.size(1)]

            outputs = self.decoder(inputs_embeds=input_embs, labels=labels)
            total_loss += outputs.loss
            num_valid += 1

        return total_loss / max(num_valid, 1)

    def compute_cpt_loss(self, text: str, s: int = 128, o: int = 128, expand_fraction: float = 0.0) -> torch.Tensor:
        """Compute next-paragraph prediction loss for CPT."""
        k = self.config.chunk_size_k
        input_ids = self._tokenize_text(text, s + o)
        ids = input_ids.squeeze(0)

        total_len = len(ids)
        if total_len < k + 4:
            return torch.tensor(0.0, device=self.device, requires_grad=True)

        actual_s = min(s, total_len // 2)
        actual_o = min(o, total_len - actual_s)
        actual_s = max(actual_s, k)
        actual_o = max(actual_o, 2)

        if actual_s + actual_o > total_len:
            actual_s = total_len - actual_o

        context_ids = ids[:actual_s]
        output_ids = ids[actual_s:actual_s + actual_o]

        if len(output_ids) < 2 or len(context_ids) < k:
            return torch.tensor(0.0, device=self.device, requires_grad=True)

        context_chunks = self._chunk_tokens(context_ids, k)
        chunk_texts = self._chunks_to_text(context_chunks)
        chunk_embeddings = self.encoder(chunk_texts, device=self.device)
        projected = self.projector(chunk_embeddings)

        if torch.isnan(projected).any() or torch.isinf(projected).any():
            return torch.tensor(0.0, device=self.device, requires_grad=True)

        context_embs = projected.unsqueeze(0)
        output_embs = self._get_decoder_embeddings(output_ids[:-1].unsqueeze(0).to(self.device))

        decoder_dtype = next(self.decoder.parameters()).dtype
        if decoder_dtype == torch.float16:
            context_embs = context_embs.clamp(-65000, 65000)
        context_embs = context_embs.to(dtype=decoder_dtype)

        full_input = torch.cat([context_embs, output_embs], dim=1)
        context_len = context_embs.size(1)
        labels = torch.cat([
            torch.full((context_len,), -100, device=self.device),
            output_ids.to(self.device)
        ]).unsqueeze(0)
        labels = labels[:, :full_input.size(1)]

        outputs = self.decoder(inputs_embeds=full_input, labels=labels)
        return outputs.loss

    def generate(self, question: str, passages: List[str], max_new_tokens: int = 256,
                 temperature: float = 0.0, use_policy: bool = True, expand_fraction: float = 0.1) -> Dict[str, Any]:
        """Generate answer using REFRAG."""
        self.eval()
        k = self.config.chunk_size_k

        context_text = " ".join(passages)
        context_ids = self._tokenize_text(context_text, self.config.max_context_tokens)
        chunks = self._chunk_tokens(context_ids, k)
        chunk_texts = self._chunks_to_text(chunks)

        with torch.no_grad():
            chunk_embeddings = self.encoder(chunk_texts, device=self.device)
            projected = self.projector(chunk_embeddings)

        L = len(chunks)
        if use_policy and L > 0:
            expand_mask, _ = self.policy.sample_expansion_mask(chunk_embeddings, expand_fraction)
        else:
            expand_mask = torch.zeros(L, dtype=torch.bool, device=self.device)

        input_parts = []
        for i, chunk_ids in enumerate(chunks):
            if expand_mask[i] if L > 0 else False:
                chunk_embs = self._get_decoder_embeddings(chunk_ids.unsqueeze(0).to(self.device))
                input_parts.append(chunk_embs.squeeze(0))
            else:
                if L > 0:
                    input_parts.append(projected[i:i+1, :])

        if input_parts:
            context_embs = torch.cat(input_parts, dim=0).unsqueeze(0)
        else:
            context_embs = torch.zeros(1, 0, self.decoder_embed_dim, device=self.device)

        q_prompt = f"Question: {question}\nAnswer:"
        q_ids = self._tokenize_text(q_prompt, self.config.max_query_tokens)
        q_embs = self._get_decoder_embeddings(q_ids)

        full_input = torch.cat([context_embs, q_embs], dim=1) if context_embs.size(1) > 0 else q_embs

        t0 = time.time()
        outputs = self.decoder(inputs_embeds=full_input, use_cache=True)
        past_kv = outputs.past_key_values
        ttft = time.time() - t0

        generated = []
        ttit_times = []
        current = torch.tensor([[self.eos_token_id]], device=self.device)

        for _ in range(max_new_tokens):
            step_emb = self._get_decoder_embeddings(current)
            t1 = time.time()
            out = self.decoder(inputs_embeds=step_emb, past_key_values=past_kv, use_cache=True)
            ttit_times.append(time.time() - t1)
            past_kv = out.past_key_values

            if temperature > 0:
                probs = F.softmax(out.logits[:, -1, :] / temperature, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
            else:
                next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)

            if next_token.item() == self.eos_token_id:
                break

            generated.append(next_token.item())
            current = next_token

        answer = self.decoder_tokenizer.decode(generated, skip_special_tokens=True)

        return {
            'answer': answer.strip(),
            'question': question,
            'num_passages': len(passages),
            'num_chunks': L,
            'num_expanded': expand_mask.sum().item() if L > 0 else 0,
            'compression_rate': k,
            'ttft_sec': ttft,
            'ttit_avg_sec': np.mean(ttit_times) if ttit_times else 0,
            'throughput_tok_per_sec': len(generated) / sum(ttit_times) if ttit_times else 0,
            'generated_tokens': len(generated),
        }

    def save_checkpoint(self, path: str, optimizer=None, scheduler=None, step: int = 0):
        """Save model checkpoint."""
        os.makedirs(path, exist_ok=True)
        checkpoint = {
            'config': self.config.to_dict(),
            'encoder_state_dict': self.encoder.state_dict(),
            'projector_state_dict': self.projector.state_dict(),
            'policy_state_dict': self.policy.state_dict(),
            'step': step,
        }
        self.decoder.save_pretrained(os.path.join(path, 'decoder'))
        if optimizer is not None:
            checkpoint['optimizer_state_dict'] = optimizer.state_dict()
        if scheduler is not None:
            checkpoint['scheduler_state_dict'] = scheduler.state_dict()
        torch.save(checkpoint, os.path.join(path, 'checkpoint.pt'))
        print(f"💾 Saved checkpoint to {path}")

    def load_checkpoint(self, path: str, load_decoder: bool = True):
        """Load model checkpoint."""
        checkpoint = torch.load(os.path.join(path, 'checkpoint.pt'), map_location=self.device)
        self.encoder.load_state_dict(checkpoint['encoder_state_dict'])
        self.projector.load_state_dict(checkpoint['projector_state_dict'], strict=False)
        self.policy.load_state_dict(checkpoint['policy_state_dict'])

        if load_decoder:
            decoder_path = os.path.join(path, 'decoder')
            if os.path.exists(decoder_path):
                self.decoder = AutoModelForCausalLM.from_pretrained(
                    decoder_path,
                    torch_dtype=torch.float16 if self.config.fp16 else torch.float32,
                ).to(self.device)
        print(f"✅ Loaded checkpoint from {path}")
        return checkpoint.get('step', 0)


print("✅ REFRAG model loaded")

In [ ]:
# @title 🏋️ Training Functions { display-mode: "form" }
# @markdown Training loops for Stage 1 (Reconstruction), Stage 2 (CPT), and Stage 3 (Policy)

def train_reconstruction(model: REFRAGModel, config: REFRAGConfig, data_path: str, output_dir: str):
    """Stage 1: Reconstruction training with curriculum learning."""
    device = get_device()
    print("=" * 50)
    print("🏋️ Stage 1: Reconstruction Training")
    print("=" * 50)
    print(f"Device: {device}")
    print(f"Config: lr={config.lr_reconstruction}, batch={config.batch_size}, stages={config.num_curriculum_stages}")

    model.freeze_decoder()
    model.train()

    dataset = CPTDataset(data_path, model.decoder_tokenizer)
    params = list(model.encoder.parameters()) + list(model.projector.parameters())
    optimizer = torch.optim.AdamW(params, lr=config.lr_reconstruction, weight_decay=config.weight_decay)
    schedule = get_curriculum_schedule(config.num_curriculum_stages)

    global_step = 0
    losses = []

    for stage_config in schedule:
        stage = stage_config['stage']
        max_chunks = stage_config['max_chunks']

        print(f"\n--- Stage {stage}/{config.num_curriculum_stages} (max_chunks={max_chunks}) ---")
        steps_per_stage = len(dataset) * config.epochs_per_stage // config.batch_size

        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(config.warmup_ratio * steps_per_stage),
            num_training_steps=steps_per_stage
        )

        running_loss = 0.0
        step_count = 0

        for epoch in range(config.epochs_per_stage):
            indices = list(range(len(dataset)))
            random.shuffle(indices)

            pbar = tqdm(range(0, len(indices), config.batch_size), desc=f"Stage {stage}", leave=False)
            for i in pbar:
                batch_indices = indices[i:i + config.batch_size]
                optimizer.zero_grad()
                batch_loss = torch.tensor(0.0, device=device, requires_grad=False)
                valid_items = 0

                for idx in batch_indices:
                    item = dataset[idx]
                    num_chunks = sample_num_chunks_for_stage(stage_config)
                    num_chunks = min(num_chunks, max_chunks)

                    loss = model.compute_reconstruction_loss(item['text'], num_chunks_cap=num_chunks)

                    if loss.requires_grad:
                        scaled_loss = loss / len(batch_indices)
                        scaled_loss.backward()
                        batch_loss = batch_loss + loss.detach()
                        valid_items += 1

                if valid_items > 0:
                    torch.nn.utils.clip_grad_norm_(params, config.max_grad_norm)
                    optimizer.step()
                    scheduler.step()

                    running_loss += (batch_loss / valid_items).item()
                    step_count += 1
                    global_step += 1
                    losses.append((batch_loss / valid_items).item())

                    if global_step % 10 == 0:
                        avg_loss = running_loss / step_count
                        lr = scheduler.get_last_lr()[0]
                        pbar.set_postfix({'loss': f'{avg_loss:.4f}', 'lr': f'{lr:.2e}'})
                        running_loss = 0.0
                        step_count = 0

    model.save_checkpoint(output_dir, optimizer, step=global_step)
    return global_step, losses


def train_cpt(model: REFRAGModel, config: REFRAGConfig, data_path: str, output_dir: str):
    """Stage 2: Continual Pre-Training (next-paragraph prediction)."""
    device = get_device()
    print("=" * 50)
    print("🏋️ Stage 2: Continual Pre-Training (CPT)")
    print("=" * 50)
    print(f"Device: {device}")
    print(f"Config: lr={config.lr_cpt}, lr_decoder={config.lr_cpt_decoder}, batch={config.batch_size}")

    model.unfreeze_decoder()
    model.train()

    dataset = CPTDataset(data_path, model.decoder_tokenizer)

    # Freeze encoder/projector, train only decoder
    for param in model.encoder.parameters():
        param.requires_grad = False
    for param in model.projector.parameters():
        param.requires_grad = False
    print("🔒 Encoder/Projector frozen for CPT")

    param_groups = [
        {'params': model.decoder.parameters(), 'lr': config.lr_cpt_decoder},
        {'params': model.policy.parameters(), 'lr': config.lr_cpt}
    ]
    optimizer = torch.optim.AdamW(param_groups, weight_decay=config.weight_decay)

    schedule = get_curriculum_schedule(config.num_curriculum_stages)
    global_step = 0
    losses = []

    for stage_config in schedule:
        stage = stage_config['stage']
        expand_frac = min(0.5, stage * 0.05)

        print(f"\n--- Stage {stage}/{config.num_curriculum_stages} (expand_frac={expand_frac:.2f}) ---")
        steps_per_stage = len(dataset) * config.epochs_per_stage // config.batch_size

        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(config.warmup_ratio * steps_per_stage),
            num_training_steps=steps_per_stage
        )

        running_loss = 0.0
        step_count = 0

        for epoch in range(config.epochs_per_stage):
            indices = list(range(len(dataset)))
            random.shuffle(indices)

            pbar = tqdm(range(0, len(indices), config.batch_size), desc=f"CPT Stage {stage}", leave=False)
            for i in pbar:
                batch_indices = indices[i:i + config.batch_size]
                optimizer.zero_grad()
                batch_loss = torch.tensor(0.0, device=device, requires_grad=False)
                valid_items = 0

                for idx in batch_indices:
                    item = dataset[idx]
                    s = config.max_context_tokens
                    o = config.max_output_tokens

                    loss = model.compute_cpt_loss(item['text'], s=s, o=o, expand_fraction=expand_frac)

                    if loss.requires_grad and not (torch.isnan(loss) or torch.isinf(loss)):
                        scaled_loss = loss / len(batch_indices)
                        scaled_loss.backward()
                        batch_loss = batch_loss + loss.detach()
                        valid_items += 1

                if valid_items > 0:
                    # Check for NaN gradients
                    has_nan_grad = False
                    for param in model.parameters():
                        if param.grad is not None and (torch.isnan(param.grad).any() or torch.isinf(param.grad).any()):
                            has_nan_grad = True
                            break

                    if has_nan_grad:
                        print(f"⚠️ NaN gradients at step {global_step}, skipping")
                        optimizer.zero_grad()
                        continue

                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)
                    optimizer.step()
                    scheduler.step()

                    running_loss += (batch_loss / valid_items).item()
                    step_count += 1
                    global_step += 1
                    losses.append((batch_loss / valid_items).item())

                    if global_step % 10 == 0:
                        avg_loss = running_loss / step_count
                        lr = scheduler.get_last_lr()[0]
                        pbar.set_postfix({'loss': f'{avg_loss:.4f}', 'lr': f'{lr:.2e}'})
                        running_loss = 0.0
                        step_count = 0

    model.save_checkpoint(output_dir, optimizer, step=global_step)
    print("✅ CPT training complete!")
    return global_step, losses


print("✅ Training functions loaded")

---

## 🚀 Training Pipeline

Now let's run the actual training pipeline. Execute the cells below in order.

In [ ]:
# @title 📊 Stage 0: Build FAISS Index { display-mode: "form" }
# @markdown Build the FAISS index from corpus for retrieval

# Check if corpus exists
if not os.path.exists(CORPUS_FILE):
    print(f"❌ Corpus file not found: {CORPUS_FILE}")
    print("   Please run the data generation cell first!")
else:
    # Load passages
    with open(CORPUS_FILE, 'r', encoding='utf-8') as f:
        passages = [line.strip() for line in f if line.strip()]
    
    print(f"📚 Loaded {len(passages)} passages from {CORPUS_FILE}")
    
    # Check if index already exists
    index_path = os.path.join(INDEX_DIR, 'faiss.index')
    if os.path.exists(index_path):
        print(f"⚠️ Index already exists at {index_path}")
        rebuild = input("Rebuild? (y/n): ").lower().strip() == 'y'
    else:
        rebuild = True
    
    if rebuild:
        print("\n🔧 Building FAISS index...")
        retriever = PassageRetriever(EMBED_MODEL)
        retriever.build_index(passages, index_path)
        print(f"✅ Index saved to {INDEX_DIR}")
    else:
        print("ℹ️ Using existing index")

In [ ]:
# @title 🔬 Initialize REFRAG Model { display-mode: "form" }
# @markdown Create the REFRAG model with configured settings

# Set seed for reproducibility
seed_everything(SEED)

# Create config
config = REFRAGConfig(
    encoder_name=ENCODER_MODEL,
    decoder_name=DECODER_MODEL,
    chunk_size_k=K,
    batch_size=BATCH_SIZE,
    num_curriculum_stages=NUM_CURRICULUM_STAGES,
    lr_reconstruction=LR_RECONSTRUCTION,
    lr_cpt=LR_CPT,
    lr_cpt_decoder=LR_CPT_DECODER,
    fp16=USE_FP16,
    seed=SEED,
)

print("🔧 Creating REFRAG model...")
print("   This may take a few minutes to download model weights...")

device = get_device()
model = REFRAGModel(config).to(device)

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n📊 Model Statistics:")
print(f"   Total parameters: {total_params / 1e6:.1f}M")
print(f"   Trainable parameters: {trainable_params / 1e6:.1f}M")
print(f"   Device: {device}")

# Check GPU memory
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    print(f"   GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

In [ ]:
# @title 🏋️ Stage 1: Reconstruction Training { display-mode: "form" }
# @markdown Train encoder + projector (decoder frozen)
# @markdown This is the first phase where we learn to compress tokens into embeddings

# Check for existing checkpoint
recon_checkpoint = os.path.join(RECON_DIR, 'checkpoint.pt')
if os.path.exists(recon_checkpoint):
    print(f"⚠️ Checkpoint exists at {RECON_DIR}")
    retrain = input("Retrain from scratch? (y/n): ").lower().strip() == 'y'
    if not retrain:
        print("ℹ️ Loading existing checkpoint...")
        model.load_checkpoint(RECON_DIR, load_decoder=False)
        print("✅ Checkpoint loaded, skipping Stage 1")
    else:
        print("🔄 Retraining from scratch...")
        import shutil
        shutil.rmtree(RECON_DIR, ignore_errors=True)
        os.makedirs(RECON_DIR, exist_ok=True)
        
        # Run training
        global_step, losses = train_reconstruction(model, config, CPT_TRAIN_FILE, RECON_DIR)
        print(f"\n✅ Stage 1 Complete! Steps: {global_step}")
else:
    print("🚀 Starting Stage 1: Reconstruction Training...")
    global_step, losses = train_reconstruction(model, config, CPT_TRAIN_FILE, RECON_DIR)
    print(f"\n✅ Stage 1 Complete! Steps: {global_step}")

In [ ]:
# @title 🏋️ Stage 2: Continual Pre-Training (CPT) { display-mode: "form" }
# @markdown Train decoder to predict next paragraph from compressed context
# @markdown This phase unfreezes the decoder and fine-tunes it

# First, ensure Stage 1 checkpoint exists
recon_checkpoint = os.path.join(RECON_DIR, 'checkpoint.pt')
if not os.path.exists(recon_checkpoint):
    print("❌ Stage 1 checkpoint not found!")
    print("   Please run Stage 1 (Reconstruction Training) first.")
else:
    # Check for existing CPT checkpoint
    cpt_checkpoint = os.path.join(CPT_DIR, 'checkpoint.pt')
    if os.path.exists(cpt_checkpoint):
        print(f"⚠️ CPT Checkpoint exists at {CPT_DIR}")
        retrain = input("Retrain from scratch? (y/n): ").lower().strip() == 'y'
        if not retrain:
            print("ℹ️ Loading existing CPT checkpoint...")
            model.load_checkpoint(CPT_DIR)
            print("✅ CPT checkpoint loaded, skipping Stage 2")
        else:
            print("🔄 Retraining CPT from Stage 1 checkpoint...")
            import shutil
            shutil.rmtree(CPT_DIR, ignore_errors=True)
            os.makedirs(CPT_DIR, exist_ok=True)
            
            # Reload Stage 1 checkpoint
            model.load_checkpoint(RECON_DIR)
            
            # Run CPT training
            global_step, losses = train_cpt(model, config, CPT_TRAIN_FILE, CPT_DIR)
            print(f"\n✅ Stage 2 (CPT) Complete! Steps: {global_step}")
    else:
        print("🚀 Starting Stage 2: Continual Pre-Training...")
        
        # Load Stage 1 checkpoint first
        print("📥 Loading Stage 1 checkpoint...")
        model.load_checkpoint(RECON_DIR)
        
        # Run CPT training
        global_step, losses = train_cpt(model, config, CPT_TRAIN_FILE, CPT_DIR)
        print(f"\n✅ Stage 2 (CPT) Complete! Steps: {global_step}")

---

## 🧪 Testing and Evaluation

Now let's test the trained model with generation and evaluation.

In [ ]:
# @title 🎯 Test Generation { display-mode: "form" }
# @markdown Test the trained model with a sample question

# Load the best checkpoint
best_checkpoint = CPT_DIR if os.path.exists(os.path.join(CPT_DIR, 'checkpoint.pt')) else RECON_DIR

if os.path.exists(os.path.join(best_checkpoint, 'checkpoint.pt')):
    print(f"📥 Loading checkpoint from: {best_checkpoint}")
    model.load_checkpoint(best_checkpoint)
    model.eval()
    
    # Load retriever
    index_path = os.path.join(INDEX_DIR, 'faiss.index')
    if os.path.exists(index_path):
        print("📥 Loading FAISS index...")
        retriever = PassageRetriever(EMBED_MODEL)
        retriever.load_index(index_path)
        
        # Test questions
        test_questions = [
            "Which river flows through City_3?",
            "What is the melting point of Alloy_11?",
            "In which field does Person_13 work?",
        ]
        
        print("\n" + "=" * 60)
        print("🧪 Testing Generation")
        print("=" * 60)
        
        for question in test_questions:
            print(f"\n❓ Question: {question}")
            
            # Retrieve passages
            results = retriever.search(question, top_k=TOP_K)
            passages = [r[0] for r in results]
            
            # Generate answer
            with torch.no_grad():
                output = model.generate(
                    question=question,
                    passages=passages,
                    max_new_tokens=64,
                    temperature=0.0,
                    use_policy=False,  # Disable policy for deterministic output
                )
            
            print(f"💬 Answer: {output['answer']}")
            print(f"   ⏱️ TTFT: {output['ttft_sec']:.3f}s | Tokens: {output['generated_tokens']}")
            print(f"   📊 Chunks: {output['num_chunks']} | Compression: {output['compression_rate']}×")
    else:
        print(f"❌ Index not found at {index_path}")
        print("   Please run Stage 0 (Build FAISS Index) first.")
else:
    print("❌ No checkpoint found!")
    print("   Please run the training stages first.")

In [ ]:
# @title 📊 Run Evaluation { display-mode: "form" }
# @markdown Evaluate the model on the test set

max_eval_samples = 50  # @param {type:"slider", min:10, max:200, step:10}

# Check prerequisites
best_checkpoint = CPT_DIR if os.path.exists(os.path.join(CPT_DIR, 'checkpoint.pt')) else RECON_DIR
index_path = os.path.join(INDEX_DIR, 'faiss.index')

if not os.path.exists(os.path.join(best_checkpoint, 'checkpoint.pt')):
    print("❌ No checkpoint found! Run training first.")
elif not os.path.exists(index_path):
    print("❌ Index not found! Run Stage 0 first.")
elif not os.path.exists(RAG_EVAL_FILE):
    print("❌ Evaluation data not found! Generate data first.")
else:
    print(f"📥 Loading checkpoint from: {best_checkpoint}")
    model.load_checkpoint(best_checkpoint)
    model.eval()
    
    print("📥 Loading FAISS index...")
    retriever = PassageRetriever(EMBED_MODEL)
    retriever.load_index(index_path)
    
    # Load eval data
    eval_data = RAGDataset(RAG_EVAL_FILE)
    
    print(f"\n🧪 Evaluating on {min(len(eval_data), max_eval_samples)} samples...")
    print("=" * 60)
    
    correct = 0
    total = 0
    total_ttft = 0.0
    total_tokens = 0
    
    results = []
    
    pbar = tqdm(eval_data.data[:max_eval_samples], desc="Evaluating")
    for item in pbar:
        question = item['question']
        gold_answers = item.get('answers', [])
        if isinstance(gold_answers, str):
            gold_answers = [gold_answers]
        
        # Retrieve and generate
        search_results = retriever.search(question, top_k=TOP_K)
        passages = [r[0] for r in search_results]
        
        with torch.no_grad():
            output = model.generate(
                question=question,
                passages=passages,
                max_new_tokens=64,
                temperature=0.0,
                use_policy=False,
            )
        
        predicted = output['answer'].lower().strip()
        
        # Check accuracy (substring match)
        is_correct = any(
            gold.lower().strip() in predicted or predicted in gold.lower().strip()
            for gold in gold_answers
        )
        
        if is_correct:
            correct += 1
        total += 1
        
        total_ttft += output['ttft_sec']
        total_tokens += output['generated_tokens']
        
        results.append({
            'question': question,
            'gold': gold_answers,
            'predicted': output['answer'],
            'correct': is_correct,
        })
        
        pbar.set_postfix({'acc': f'{correct/total:.2%}'})
    
    # Print results
    print("\n" + "=" * 60)
    print("📊 Evaluation Results")
    print("=" * 60)
    print(f"   Accuracy: {correct}/{total} ({correct/total:.2%})")
    print(f"   Avg TTFT: {total_ttft/total:.4f}s")
    print(f"   Avg tokens: {total_tokens/total:.1f}")
    
    # Show some examples
    print("\n📝 Sample Results:")
    for i, r in enumerate(results[:5]):
        status = "✅" if r['correct'] else "❌"
        print(f"   {status} Q: {r['question'][:50]}...")
        print(f"      Gold: {r['gold'][0] if r['gold'] else 'N/A'}")
        print(f"      Pred: {r['predicted'][:50]}")

---

## 💾 Save and Download Model

Save the trained model and download it from Colab.

In [ ]:
# @title 💾 Save Model to Google Drive (Colab) { display-mode: "form" }
# @markdown Mount Google Drive and save the model checkpoint

try:
    from google.colab import drive
    
    # Mount Drive
    print("📁 Mounting Google Drive...")
    drive.mount('/content/drive')
    
    # Create save directory
    save_dir = '/content/drive/MyDrive/refrag_v2_checkpoints'
    os.makedirs(save_dir, exist_ok=True)
    
    # Copy checkpoints
    import shutil
    from datetime import datetime
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    for src_dir, name in [(RECON_DIR, 'recon'), (CPT_DIR, 'cpt'), (INDEX_DIR, 'index')]:
        if os.path.exists(src_dir):
            dst_dir = os.path.join(save_dir, f'{name}_{timestamp}')
            print(f"💾 Copying {src_dir} -> {dst_dir}")
            shutil.copytree(src_dir, dst_dir)
    
    print(f"\n✅ Model saved to Google Drive: {save_dir}")
    
except ImportError:
    print("ℹ️ Not running in Colab. Checkpoints are saved locally in 'runs/' directory.")

In [ ]:
# @title 📥 Download Model (Colab) { display-mode: "form" }
# @markdown Download the model checkpoint as a zip file

try:
    from google.colab import files
    import shutil
    import zipfile
    
    # Create a zip file of the checkpoints
    zip_filename = 'refrag_v2_checkpoint.zip'
    
    print("📦 Creating zip archive...")
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root_dir in [RECON_DIR, CPT_DIR, INDEX_DIR]:
            if os.path.exists(root_dir):
                for root, dirs, files_list in os.walk(root_dir):
                    for file in files_list:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path, os.path.dirname(root_dir))
                        zipf.write(file_path, arcname)
                        print(f"   Added: {arcname}")
    
    # Get file size
    zip_size = os.path.getsize(zip_filename) / 1e6
    print(f"\n📊 Zip file size: {zip_size:.1f} MB")
    
    print("⬇️ Starting download...")
    files.download(zip_filename)
    
except ImportError:
    print("ℹ️ Not running in Colab. Checkpoints are in 'runs/' directory.")
    print(f"   - Reconstruction: {RECON_DIR}")
    print(f"   - CPT: {CPT_DIR}")
    print(f"   - Index: {INDEX_DIR}")

---

## 🎮 Interactive Demo

Try the model with your own questions!

In [ ]:
# @title 🎮 Interactive Question Answering { display-mode: "form" }
# @markdown Enter your own question to test the model

your_question = "Which river flows through City_10?"  # @param {type:"string"}
max_tokens = 64  # @param {type:"slider", min:16, max:256, step:16}
temperature = 0.0  # @param {type:"slider", min:0.0, max:1.0, step:0.1}

# Check prerequisites
best_checkpoint = CPT_DIR if os.path.exists(os.path.join(CPT_DIR, 'checkpoint.pt')) else RECON_DIR
index_path = os.path.join(INDEX_DIR, 'faiss.index')

if not os.path.exists(os.path.join(best_checkpoint, 'checkpoint.pt')):
    print("❌ No checkpoint found! Run training first.")
elif not os.path.exists(index_path):
    print("❌ Index not found! Run Stage 0 first.")
else:
    # Ensure model and retriever are loaded
    if 'retriever' not in dir() or retriever is None:
        print("📥 Loading retriever...")
        retriever = PassageRetriever(EMBED_MODEL)
        retriever.load_index(index_path)
    
    model.eval()
    
    print(f"\n❓ Question: {your_question}")
    print("-" * 50)
    
    # Retrieve passages
    search_results = retriever.search(your_question, top_k=TOP_K)
    passages = [r[0] for r in search_results]
    
    print(f"📚 Retrieved {len(passages)} passages:")
    for i, (passage, score) in enumerate(search_results[:3]):
        print(f"   [{i+1}] (score: {score:.3f}) {passage[:80]}...")
    
    # Generate answer
    with torch.no_grad():
        output = model.generate(
            question=your_question,
            passages=passages,
            max_new_tokens=max_tokens,
            temperature=temperature,
            use_policy=False,
        )
    
    print(f"\n💬 Answer: {output['answer']}")
    print(f"\n📊 Statistics:")
    print(f"   ⏱️ Time to first token: {output['ttft_sec']:.3f}s")
    print(f"   📝 Generated tokens: {output['generated_tokens']}")
    print(f"   📦 Chunks used: {output['num_chunks']}")
    print(f"   🗜️ Compression rate: {output['compression_rate']}×")

---

## 📖 Appendix: Troubleshooting & Tips

### Common Issues

1. **Out of Memory (OOM)**
   - Reduce `BATCH_SIZE` to 1-2
   - Use a smaller decoder model (e.g., `TinyLlama/TinyLlama-1.1B-Chat-v1.0`)
   - Reduce `NUM_CURRICULUM_STAGES` to 3-5
   
2. **NaN Loss during training**
   - Set `USE_FP16 = False`
   - Reduce learning rates
   
3. **Slow training**
   - Ensure you're using a GPU (check Runtime > Change runtime type)
   - Use Colab Pro for better GPUs (A100, V100)

4. **HuggingFace authentication errors**
   - Make sure you've accepted the model license on HuggingFace
   - Try re-running the login cell

### GPU Memory Guide

| Model | GPU Memory | Recommended Batch Size |
|-------|------------|------------------------|
| TinyLlama-1.1B | ~4GB | 4-8 |
| Llama-3.2-1B | ~6GB | 2-4 |
| Llama-3.2-3B | ~12GB | 1-2 |
| Phi-2 | ~8GB | 2-4 |

### Paper Reference

REFRAG: Rethinking RAG based Decoding  
Meta Superintelligence Labs  
arXiv:2509.01092v2